# 使用CUDA Math API，用蒙特卡洛方法计算定积分

In [1]:
import numpy as np
from pycuda.compiler import SourceModule
import pycuda.driver as cuda
import pycuda.autoinit
import time

我们使用CUDA Math API，运用蒙特卡洛方法计算定积分$$\int_{-10}^{10}\int_{-10}^{10}e^{-(x^2+y^2)}dxdy,$$并与在CPU上使用蒙特卡洛方法计算定积分的速度进行比较。

首先使用CUDA Math API进行计算。

In [2]:
# CUDA 内核代码
cuda_code = """
#include <curand_kernel.h>
#include <math.h>

// CUDA 核函数：使用蒙特卡洛方法计算定积分
extern "C" __global__ void monte_carlo(float* results, int samples_per_thread, int seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;  // 线程索引
    curandState state;
    curand_init(seed, idx, 0, &state);  // 初始化 cuRAND 随机数状态

    float local_sum = 0.0f;

    for (int i = 0; i < samples_per_thread; i++) {
        // 生成 x 和 y，均匀分布在 [-10, 10]
        float x = -10.0f + 20.0f * curand_uniform(&state);
        float y = -10.0f + 20.0f * curand_uniform(&state);

        // 使用 CUDA Math API 计算 f(x, y) = exp(-x^2 - y^2)
        local_sum += __expf(-x * x - y * y);
    }

    // 将局部和写回全局内存
    results[idx] = local_sum;
}

"""

# 主函数
def monte_carlo_cuda(samples_per_thread=1000, threads_per_block=256, blocks=64):
    # 总线程数
    total_threads = threads_per_block * blocks
    total_samples = samples_per_thread * total_threads

    # 分配 GPU 内存
    results_device = cuda.mem_alloc(total_threads * np.float32().nbytes)

    # 编译 CUDA 内核
    mod = SourceModule(no_extern_c=True,options=['-w'],source=cuda_code)
    monte_carlo_kernel = mod.get_function("monte_carlo")

    # 启动核函数
    start_time = time.time()
    monte_carlo_kernel(
        results_device,
        np.int32(samples_per_thread),
        np.int32(int(time.time())),
        block=(threads_per_block, 1, 1),
        grid=(blocks, 1),
    )
    cuda.Context.synchronize()
    end_time = time.time()

    # 拷贝结果回主机
    results_host = np.empty(total_threads, dtype=np.float32)
    cuda.memcpy_dtoh(results_host, results_device)

    # 归约求和
    integral_sum = np.sum(results_host)

    # 计算最终结果
    area = 400.0  # 积分区域面积 [-10, 10] x [-10, 10]
    result = (integral_sum / total_samples) * area

    print(f"积分估计值: {result}")
    print(f"样本点数: {total_samples}")
    print(f"用时: {end_time - start_time:.4f}秒")

# 运行程序
if __name__ == "__main__":
    monte_carlo_cuda()

积分估计值: 3.1401420593261715
样本点数: 16384000
用时: 0.0014秒


然后使用CPU进行计算。

In [3]:
def monte_carlo_cpu(samples_per_thread=1000, total_threads=16384):
    start = time.time()
    # 总样本点数
    total_samples = samples_per_thread * total_threads

    # 初始化随机数种子
    np.random.seed(int(time.time()))

    # 生成随机样本点 (x, y) 均匀分布在 [-10, 10]
    x_samples = np.random.uniform(-10.0, 10.0, total_samples)
    y_samples = np.random.uniform(-10.0, 10.0, total_samples)

    # 计算 f(x, y) = exp(-x^2 - y^2)
    function_values = np.exp(-x_samples**2 - y_samples**2)

    # 归约求和
    integral_sum = np.sum(function_values)

    # 计算最终结果
    area = 400.0  # 积分区域面积 [-10, 10] x [-10, 10]
    result = (integral_sum / total_samples) * area
    
    end = time.time()
    
    print(f"积分估计值: {result}")
    print(f"样本点数: {total_samples}")
    print(f"用时: {end - start:.4f}秒")

# 运行程序
if __name__ == "__main__":
    monte_carlo_cpu()

积分估计值: 3.1307765623002406
样本点数: 16384000
用时: 0.6788秒
